In [211]:
import pandas as pd
import sklearn
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [5]:
path = "C:/Users/ericluo04/Documents/GitHub/Bots-Project/Code/"

In [207]:
df = pd.read_csv(path + "2. HK Training/labeling/Data/labeled_data_balanced.csv")
df['tweet'] = df['tweet'].apply(nltk.word_tokenize)
stemmer = PorterStemmer()
df['tweet'] = df['tweet'].apply(lambda x: [stemmer.stem(y) for y in x])
df['tweet'] = df['tweet'].apply(lambda x: ' '.join(x))

In [242]:
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['label'], train_size=.8)

stops = stopwords.words('english')
cv = CountVectorizer(strip_accents='unicode', stop_words=frozenset(stops + ['hong', 'kong', 'hongkong', 'thi', 'rt', 'thank', 'hk', 'hongkongprotest', 'hkprotest', 'u2026', 'uff0c', 'http', 'co']), min_df=2)
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [243]:
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train_cv, y_train)
predictions = naive_bayes.predict(X_test_cv)

print('Accuracy score: ', accuracy_score(y_test, predictions))
print('Precision score: ', precision_score(y_test, predictions))
print('Recall score: ', recall_score(y_test, predictions))

Accuracy score:  0.9204301075268817
Precision score:  0.9354838709677419
Recall score:  0.9049645390070922


In [246]:
neg_class_prob_sorted = naive_bayes.feature_log_prob_[0, :].argsort()
pos_class_prob_sorted = naive_bayes.feature_log_prob_[1, :].argsort()

print('These are the 10 most predictive features for a pro-Hong Kong tweet:', np.take(cv.get_feature_names(), neg_class_prob_sorted[-10:]))
print('These are the 10 most predictive features for a pro-China tweet:', np.take(cv.get_feature_names(), pos_class_prob_sorted[-10:]))

These are the 10 most predictive features for a pro-Hong Kong tweet: ['freedom' 'support' 'standwithhk' 'nba' 'china' 'fightforfreedom' 'polic'
 'freehongkong' 'chinazi' 'standwithhongkong']
These are the 10 most predictive features for a pro-China tweet: ['u7684' 'u66b4' 'hkriot' 'hongkongriot' 'polic' 'rioter' 'u9999' 'u6e2f'
 'stophkriot' 'hkrioter']
